In [177]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")

In [178]:
data_raw = pd.read_csv('PROJECT/COVID-19 Philippines.csv')
data_clean = data_raw.iloc[:,0:25]
data_clean = data_clean.dropna(how='all')

# Epi Link

In [179]:
data_clean['Epi_Link'].value_counts()

With Exposure to known COVID-19 case                                                                                                                                                                                                               8
With exposure to known COVID-19 case                                                                                                                                                                                                               4
Related to a known COVID-19 case                                                                                                                                                                                                                   3
With exposure to PH21                                                                                                                                                                                                                              2
Contact of PH43     

In [180]:
#PREP
#make everything lowercase so words like Travel and travel are equivalent
data_clean['Epi_Link']=data_clean['Epi_Link'].str.lower()

#make NaNs strings since we are using boolean string comparison and I was getting weird errors
data_clean['Epi_Link']=data_clean['Epi_Link'].fillna("NaN") 

Flag for people who traveled <br>
- Approach: If epi_link contains the word travel, epi_link_traveled == 1

In [181]:
#travel flag
data_clean['Epi_Link_Traveled']=0
for i in range(len(data_clean)):
    if'travel' in data_clean['Epi_Link'][i]:
        data_clean['Epi_Link_Traveled'][i] = 1 

In [182]:
data_clean[data_clean['Epi_Link_Traveled']==1]

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Health Status,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long,Epi_Link_Traveled
2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,22-Jan-2020,...,Recovered,15-Mar-2020,NaN,51 - 60,Manila City,14.615867,120.980991,14.598727,120.981991,1
261,PH00262,M,45.0,Filipino,"Davao City, Davao Del Sur",Came from Manila,For validation,"husband of ph263, both traveled from manila",NaN,NaN,...,Mild,NaN,NaN,41 - 50,"Davao City, Davao Del Sur",14.615867,120.980991,7.188425,125.447792,1
262,PH00263,F,46.0,Filipino,"Davao City, Davao Del Sur",Came from Manila,For validation,"wife of ph262, both traveled from manila",NaN,NaN,...,Mild,NaN,NaN,41 - 50,"Davao City, Davao Del Sur",7.099528,125.619416,7.188425,125.447792,1
441,PH00442,M,28.0,For validation,"Davao City, Davao Del Sur",For validation,For validation,traveled from manila,NaN,NaN,...,For validation,NaN,NaN,21 - 30,"Davao City, Davao Del Sur",7.099528,125.619416,7.188425,125.447792,1
442,PH00443,M,32.0,Filipino,"Davao City, Davao Del Sur",For validation,For validation,traveled from manila,NaN,NaN,...,For validation,NaN,NaN,31 - 40,"Davao City, Davao Del Sur",14.615867,120.980991,7.188425,125.447792,1
443,PH00444,M,50.0,Filipino,Makati City,For validation,For validation,"traveled from manila, palawan, cebu",NaN,NaN,...,For validation,NaN,NaN,41 - 50,Makati City,7.422318,125.828438,14.554498,121.022931,1


Exposure count
- Approach: For instances that aren't about travel or aren't really long (less than 50 characters), split on the comma, colon, or and and count

In [183]:
#epi link exposure
data_clean['Epi_Link_Exposure']=0
for i in range(len(data_clean)):
    if len(data_clean['Epi_Link'][i]) < 50:
        data_clean['Epi_Link'][i]=re.split('; |, | and', data_clean['Epi_Link'][i])                                    
        data_clean['Epi_Link_Exposure'][i] = len(data_clean['Epi_Link'][i])
                                           
    #for now ignore instances that are about travel
    if'travel' in data_clean['Epi_Link'][i]:
        data_clean['Epi_Link_Exposure'][i] = 0 

In [176]:
data_clean.head(10)

,Case No.,Sex,Age,Nationality,Residence in the Philippines,History of Travel / Exposure,Travel History,Epi_Link,Date of Onset of Symptoms,Date of Admission,...,Date of Final Status (recovered/expired),Final Diagnosis,Age Group,Location,Latitude,Longitude,Residence Lat,Residence Long,Epi_Link_Traveled,Epi_Link_Exposure
0,PH00001,F,38.0,Chinese,None,Yes (China); Wife of PH02,China,[wife of ph2],21-Jan-20,25-Jan-2020,...,15-Mar-2020,NaN,31 - 40,Manila City,14.613480,120.980950,14.598727,120.981991,0,1
1,PH00002,M,44.0,Chinese,None,Yes (China); Husband of PH1,China,[husband of ph1],18-Jan-20,25-Jan-2020,...,1-Feb-2020,Severe Pneumonia,41 - 50,Manila City,14.613480,120.980950,14.598727,120.981991,0,1
2,PH00003,F,60.0,Chinese,None,Yes (China),China,traveled to cebu and bohol. was already in chi...,21-Jan-20,22-Jan-2020,...,15-Mar-2020,NaN,51 - 60,Manila City,14.615867,120.980991,14.598727,120.981991,1,0
3,PH00004,M,48.0,Filipino,Taguig City,Yes (Japan),Japan,[NaN],03-Mar-20,6-Mar-2020,...,19-Mar-2020,"Asymptomatic, Negative Test Result",41 - 50,Taguig City,14.606904,121.020392,14.517274,121.049613,0,1
4,PH00005,M,62.0,Filipino,"Cainta, Rizal",Husband of PH06,None,[husband of ph6],25-Feb-20,5-Mar-2020,...,12-Mar-2020,"Acute Respiratory Distress Syndrome, Severe pn...",61 - 70,"Cainta, Rizal",14.409635,121.037425,14.586506,121.113825,0,1
5,PH00006,F,59.0,Filipino,"Cainta, Rizal",Wife of PH05,None,[wife of ph5],27-Feb-20,5-Mar-2020,...,11-Mar-2020,"Acute Respiratory Distress Syndrome, Severe pn...",51 - 60,"Cainta, Rizal",14.409635,121.037425,14.586506,121.113825,0,1
6,PH00007,M,38.0,"Chinese, Taiwanese",Makati City,No Travel History; Contact of FN2,None,[contact of fn2],03-Mar-20,6-Mar-2020,...,21-Mar-2020,"Asymptomatic, Negative Test Result",31 - 40,Makati City,14.409635,121.037425,14.554498,121.022931,0,1
7,PH00008,M,32.0,Filipino,Pasig City,Yes (Japan),Japan,[NaN],01-Mar-20,8-Mar-2020,...,NaN,NaN,31 - 40,Pasig City,14.555719,121.048688,14.576163,121.084615,0,1
8,PH00009,M,86.0,American,Marikina City,"Yes (USA, Korea); In same household with PH027...","USA, Korea","[in same household with ph27-30, contact of ph31]",01-Mar-20,6-Mar-2020,...,14-Mar-2020,"Multiple Organ Failure, Septic Shock, Acute Re...",81 - 90,Marikina City,14.589620,121.069797,14.650423,121.102348,0,2
9,PH00010,M,57.0,Filipino,Quezon City,Taiwan,Taiwan,case reported that a relative from china (with...,20-Feb-20,7-Mar-2020,...,23-Mar-2020,"Asymptomatic, Negative Test Result",51 - 60,Quezon City,14.613480,120.980950,14.676045,121.043361,0,0


# Dates

In [ ]:
diff_admit_symp

In [ ]:
diff_final_admit

In [ ]:
diff_final_symp